In [ ]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_openai_functions_agent, AgentExecutor, load_tools
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables import Runnable
from langchain_openai import AzureChatOpenAI
from langchain.agents import initialize_agent, Tool, AgentType

# ツールの定義（整数や文字列入力を受け付け）
def prime_factors(n):
    n = int(n)
    factors = []
    divisor = 2
    while divisor * divisor <= n:
        while n % divisor == 0:
            factors.append(divisor)
            n //= divisor
        divisor += 1
    if n > 1:
        factors.append(n)
    return factors

# 合計ツール
def sum_of_numbers(numbers):
    if isinstance(numbers, list):
        # すでにリストならそのまま合計
        return sum(numbers)
    
    elif isinstance(numbers, str):
        # [] を除去して空白を除き、数式として評価を試みる
        cleaned = numbers.strip("[]").replace(" ", "")
        
        # 式として評価できる場合（例: 2+3+4）
        try:
            return eval(cleaned)
        except Exception:
            pass
        
        # カンマ区切りとして処理（例: "2,3,4"）
        try:
            nums = [int(x.strip()) for x in cleaned.split(",")]
            return sum(nums)
        except Exception:
            raise ValueError(f"String形式の解釈に失敗しました: {numbers}")
    
    else:
        raise ValueError("Invalid input type for sum_of_numbers. Supported types: list or str")

# LLM 初期化
llm = AzureChatOpenAI(
    azure_endpoint="https://weida-mbw67lla-swedencentral.cognitiveservices.azure.com/",
    azure_deployment="gpt-4.1",
    api_version="2024-12-01-preview",
    temperature=0.7
)

# ツール登録
tools = [
    Tool(name="prime_factors", func=prime_factors, description="Factorize a given integer"),
    Tool(name="sum_of_numbers", func=sum_of_numbers, description="Sum a list of numbers"),
]+load_tools(["llm-math"], llm = llm)

# メモリ定義
memory = ConversationBufferMemory(
    return_messages=True,
    memory_key="chat_history"
)

# Promptテンプレート（3つのinput: input, chat_history, agent_scratchpad）
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# Agentの作成
# Agentの種類：AgentType.OPENAI_FUNCTIONS
# 特徴：OpenAIのFunction Callingに対応。LangChainが関数の選択と入力の構築を自動化。
# 使用LLM：ChatOpenAI / AzureChatOpenAI など Function対応モデル
agent = create_openai_functions_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

#複雑な推論やステップバイステップが必要	create_react_agent
#情報検索や調査系の質問を扱いたい	create_self_ask_with_search_agent

# AgentExecutorの作成
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True
)

# ==========================
# 対話モード：Consoleから入力
# ==========================
print("対話モードを開始します。'exit' または '終了' と入力すると終了します。")

while True:
    user_input = input("あなた: ")

    if user_input.lower() in {"exit", "quit", "終了"}:
        print("対話モードを終了します。")
        break

    try:
        response = agent_executor.invoke({"input": user_input})
        print("アシスタント:", response["output"])
    except Exception as e:
        print(f"エラーが発生しました: {e}")

# # 実行
# resp1 = agent_executor.invoke({"input": "360を素因数分解したときの、すべての素因数の総和を求めてください。"})
# print(resp1["output"])

# resp2 = agent_executor.invoke({"input": "その算出した数を2倍にしてほしいです。関数使わずに"})
# print(resp2["output"])

# resp3 = agent_executor.invoke({"input": "その算出した数を0.43乗してほしいです。"})
# print(resp3["output"])

# #入力形式が結構ランダム
# #ex)二つ目のinvokingで引数に[2,2,3,3,5]入れたり、[2+2+3+3+5]入れたり...
# print(memory.load_memory_variables({}))

対話モードを開始します。'exit' または '終了' と入力すると終了します。


あなた:  234を素因数分解して、その素因数の合計を計算し、その値を2倍して、再度素因数分解してほしいです。




> Entering new AgentExecutor chain...

Invoking: `prime_factors` with `234`


[2, 3, 3, 13]
Invoking: `sum_of_numbers` with `2,3,3,13`


(2, 3, 3, 13)
Invoking: `Calculator` with `2+3+3+13`


Answer: 21
Invoking: `Calculator` with `21*2`


Answer: 42
Invoking: `prime_factors` with `42`


[2, 3, 7]1. 234の素因数分解は「2, 3, 3, 13」です。
2. これらの素因数の合計は 2 + 3 + 3 + 13 = 21 です。
3. その値を2倍すると 21 × 2 = 42 になります。
4. 42を素因数分解すると「2, 3, 7」です。

まとめ：
- 234 → 素因数「2, 3, 3, 13」
- 合計 → 21
- 2倍 → 42
- 42の素因数分解 → 「2, 3, 7」

> Finished chain.
アシスタント: 1. 234の素因数分解は「2, 3, 3, 13」です。
2. これらの素因数の合計は 2 + 3 + 3 + 13 = 21 です。
3. その値を2倍すると 21 × 2 = 42 になります。
4. 42を素因数分解すると「2, 3, 7」です。

まとめ：
- 234 → 素因数「2, 3, 3, 13」
- 合計 → 21
- 2倍 → 42
- 42の素因数分解 → 「2, 3, 7」


あなた:  あなたの名前は？




> Entering new AgentExecutor chain...
私はAIアシスタントです。特定の名前はありませんが、「アシスタント」や「チャットボット」と呼んでいただいて大丈夫です。もし好きな名前を付けたい場合は、お好きな名前で呼んでください！

> Finished chain.
アシスタント: 私はAIアシスタントです。特定の名前はありませんが、「アシスタント」や「チャットボット」と呼んでいただいて大丈夫です。もし好きな名前を付けたい場合は、お好きな名前で呼んでください！


...



> Entering new AgentExecutor chain...

Invoking: `prime_factors` with `2394092830`


[2, 5, 17, 887, 15877]
Invoking: `sum_of_numbers` with `2, 5, 17, 887, 15877`


(2, 5, 17, 887, 15877)
Invoking: `Calculator` with `2 + 5 + 17 + 887 + 15877`


Answer: 16788
Invoking: `prime_factors` with `16788`


[2, 2, 3, 1399]
Invoking: `sum_of_numbers` with `2, 2, 3, 1399`


(2, 2, 3, 1399)
Invoking: `Calculator` with `2 + 2 + 3 + 1399`


Answer: 1406
Invoking: `prime_factors` with `1406`


[2, 19, 37]
Invoking: `sum_of_numbers` with `2, 19, 37`


(2, 19, 37)
Invoking: `Calculator` with `2 + 19 + 37`


Answer: 58
Invoking: `prime_factors` with `58`


[2, 29]
Invoking: `sum_of_numbers` with `2, 29`


(2, 29)
Invoking: `Calculator` with `2 + 29`


Answer: 31
Invoking: `prime_factors` with `31`


[31]
Invoking: `sum_of_numbers` with `31`


31
Invoking: `prime_factors` with `31`


[31]

> Finished chain.
アシスタント: Agent stopped due to iteration limit or time limit.


あなた:  なぜ止まったのでしょうか？もう素数になってしまい、これ以上続けても意味ないからでしょうか？




> Entering new AgentExecutor chain...
ご質問ありがとうございます。先ほど処理が途中で止まった理由は、私（サリ）の内部的な処理制限（繰り返し回数や時間制限）があったためです。  
おっしゃる通り「素数になってしまったから」ではなく、技術的な制限によるものです。

**補足説明：**  
「素因数分解→素因数の和→その和を再度素因数分解…」という操作は、和が素数になれば次のステップで素因数分解してもその数自身しか出てこなくなり、以降は同じ数の繰り返しになります。しかし、前回止まったのは、その前に内部の繰り返し（ループ）回数制限や処理時間上限に達したためです。

もしご希望であれば、回数を減らして（例：5回や10回など）もう一度試すこともできますし、途中までの結果を順番に表示することもできます。どうされますか？

> Finished chain.
アシスタント: ご質問ありがとうございます。先ほど処理が途中で止まった理由は、私（サリ）の内部的な処理制限（繰り返し回数や時間制限）があったためです。  
おっしゃる通り「素数になってしまったから」ではなく、技術的な制限によるものです。

**補足説明：**  
「素因数分解→素因数の和→その和を再度素因数分解…」という操作は、和が素数になれば次のステップで素因数分解してもその数自身しか出てこなくなり、以降は同じ数の繰り返しになります。しかし、前回止まったのは、その前に内部の繰り返し（ループ）回数制限や処理時間上限に達したためです。

もしご希望であれば、回数を減らして（例：5回や10回など）もう一度試すこともできますし、途中までの結果を順番に表示することもできます。どうされますか？


あなた:  繰り返し制限や処理時間って引数等で変えられますか？




> Entering new AgentExecutor chain...
良いご質問ありがとうございます。

現時点（2024年6月時点）で、ChatGPTのようなAIアシスタントが用いている繰り返し制限や処理時間の設定は、ユーザー側が引数やコマンドで変更することはできません。  
これらはシステム側で安全性や安定動作のために設定されている上限（例：一度のリクエストでの最大ループ回数や時間）なので、ユーザーが直接カスタマイズすることはできない仕様です。

### 補足
- もし大量の計算や長い処理をしたい場合は、回数を減らす、または途中で区切って何度かに分けて質問する形になります。
- プログラムを書ける場合は、自分のPCやクラウドサービス上でスクリプトを実行することで制限なく計算することも可能です（たとえばPythonなど）。

もし途中までの結果を知りたい、または何回かに分けて計算したい場合はご希望をお聞かせください。できる範囲でサポートいたします！

> Finished chain.
アシスタント: 良いご質問ありがとうございます。

現時点（2024年6月時点）で、ChatGPTのようなAIアシスタントが用いている繰り返し制限や処理時間の設定は、ユーザー側が引数やコマンドで変更することはできません。  
これらはシステム側で安全性や安定動作のために設定されている上限（例：一度のリクエストでの最大ループ回数や時間）なので、ユーザーが直接カスタマイズすることはできない仕様です。

### 補足
- もし大量の計算や長い処理をしたい場合は、回数を減らす、または途中で区切って何度かに分けて質問する形になります。
- プログラムを書ける場合は、自分のPCやクラウドサービス上でスクリプトを実行することで制限なく計算することも可能です（たとえばPythonなど）。

もし途中までの結果を知りたい、または何回かに分けて計算したい場合はご希望をお聞かせください。できる範囲でサポートいたします！


In [ ]:
print(memory.load_memory_variables({}))